# Pseudo-compressible Navier-Stokes element symbolic formulation and implementation
**Rubén Zorrilla, Riccardo Rossi**
UPC BarcelonaTech, CIMNE

The Navier-Stokes equations are conformed by the linear momentum conservation
equation

$
\frac{\partial\rho\mathbf{u}}{\partial t}-\nabla\cdot\mathbf{\boldsymbol{\sigma}}+\rho\mathbf{a}\cdot\nabla\mathbf{u}=\rho\mathbf{f}
$

and the mass conservation equation
$
\frac{D\rho}{Dt}+\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)=0
$

Developing the material time derivative of the mass conservation equation
one has

$
\frac{\partial\rho}{\partial t}+\mathbf{u\cdot\nabla\rho}+\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)=0
$

Let us assume small space variations of the density field, yielding

$
\frac{\partial\rho}{\partial t}+\rho\nabla\cdot\mathbf{u}=0
$

The density local time derivative can be obtained as

$
\frac{\partial p}{\partial t}=\frac{\partial p}{\partial\rho}\frac{\partial\rho}{\partial t}
$

$
\frac{\partial\rho}{\partial t}=\left(\frac{\partial p}{\partial\rho}\right)^{-1}\frac{\partial p}{\partial t}
$

Considering the state equation

$
\frac{\partial p}{\partial\rho}=c^{2}
$

the mass conservation equation can be rewritten as

$
\frac{1}{c^{2}}\frac{\partial p}{\partial t}+\rho\nabla\cdot\mathbf{u}=0
$

where c is the sound velocity in the considered fluid. Once arrived
to this point, one can divide the previous equation by $\rho$ for
the sake of having a better conditioned matrix (specially in those
cases in where $\rho\gg1$). This has been implemented as an user-defined
option in the symbolic element generation script. Note that this will
also affect the stabilization terms.

On the other hand, the density is considered to be time independent
in the momentum conservation equation, thus
$
\rho\frac{\partial\mathbf{u}}{\partial t}-\nabla\cdot\mathbf{\boldsymbol{\sigma}}+\rho\mathbf{a}\cdot\nabla\mathbf{u}=\rho\mathbf{f}
$

In addition, the stress tensor $\boldsymbol{\sigma}$ is defined as 

$
\boldsymbol{\sigma}=-p\mathbf{I}+\rho(\nabla\cdot\mathbf{u})\mathbf{I}+\boldsymbol{\tau}
$

Taking into account that no volumetric deformation is considered in
the momentum conservation equation, the stress tensor $\boldsymbol{\sigma}$
to be considered turns to be

$
\boldsymbol{\sigma}=-p\mathbf{I}+\boldsymbol{\tau}
$

where $p$ is the pressure and $\boldsymbol{\tau}$ the shear stress
tensor, which value and computation depends on the constitutive tensor
employed as $\boldsymbol{\tau}=\mathbb{\mathbb{C}}\nabla^{s}\mathbf{u}$,
being $\nabla^{s}$ the symmetric gradient operator.
Substituting the stress tensor above, the linear momentum equation
reads as follows

$
\rho\frac{\partial\mathbf{u}}{\partial t}-\nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right)+\rho\mathbf{a}\cdot\nabla\mathbf{u}=\rho\mathbf{f}
$

Once arrived to this point, one can define the linear momentum and
mass conservation residuals as

$
\mathbf{R}^{M}(\mathbf{u},p)=\rho\mathbf{f}-\rho\frac{\partial\mathbf{u}}{\partial t}+\nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right)-\rho\mathbf{a}\cdot\nabla\mathbf{u}
$

$
\mathbf{R}^{C}(\mathbf{u},p)=-\frac{1}{c^{2}}\frac{\partial p}{\partial t}-\rho\nabla\cdot\mathbf{u}
$

## Galerkin weak form and ASGS stabilization

Considering the velocity test function $\mathbf{w}$ and pressure
test function $q$, the Galerkin weak form of the problem can be obtained
as

$
\left(\mathbf{w},\mathbf{R}^{M}(\mathbf{u},p)\right)_{\Omega}+\left(q,\mathbf{R}^{C}(\mathbf{u},p)\right)_{\Omega}=0
$

$
\left(\mathbf{w},\rho\mathbf{f}-\rho\frac{\partial\mathbf{u}}{\partial t}+\nabla\cdot\left(-p\mathbf{I}+\boldsymbol{\tau}\right)-\rho\mathbf{a}\cdot\nabla\mathbf{u}\right)_{\Omega}+\left(q,-\frac{1}{c^{2}}\frac{\partial p}{\partial t}-\rho\nabla\cdot\mathbf{u}\right)_{\Omega}=0
$

On top of that, the following multiscale decomposition is considered

\begin{eqnarray*}
\mathbf{u} & = & \mathbf{u_{h}}+\mathbf{u_{s}}\\
p & = & p_{h}+p_{s}
\end{eqnarray*}

where $\mathbf{u_{\mathbf{h}}}$ and $p_{h}$ belong to the finite
element space while $\mathbf{u_{s}}$ and $p_{s}$ are the so called
unresolvable subscales. Substituting the proposed decomposition in
the Galerkin residual weak form above yields

$
\left(\mathbf{w},\rho\mathbf{f}-\rho\frac{\partial(\mathbf{u_{h}}+\mathbf{u_{s}})}{\partial t}+\nabla\cdot\left(-(p_{h}+p_{s})\mathbf{I}+\mathbb{\mathbb{C}}\nabla^s(\mathbf{u_{h}}+\mathbf{u_{s}})\right)-\rho\mathbf{a}\cdot\nabla(\mathbf{u_{h}}+\mathbf{u_{s}})\right)_{\Omega}+\left(q,-\frac{1}{c^{2}}\frac{\partial(p_{h}+p_{s})}{\partial t}-\rho\nabla\cdot(\mathbf{u_{h}}+\mathbf{u_{s}})\right)_{\Omega}=0
$

Once arrived to this point it is important to clearly state the assumptions
that are to be taken from now on


- Quasi-static subscales: $\frac{\partial\mathbf{u_{s}}}{\partial t}=\frac{\partial p_{s}}{\partial t}\approx0$
- $\mathbf{u_{s}}\approx p_{s}\approx0$ on $\Gamma$
- Linear order finite elements (higher order derivatives are equal to
0)


Developing terms in the decomposed Galerkin residual weak form considering
the quasi-static nature of the subscales one obtains

\begin{align*}
\left(\mathbf{w},\rho\mathbf{f}\right)_{\Omega}-\left(\mathbf{w},\rho\frac{\partial\mathbf{u_{h}}}{\partial t}\right)_{\Omega}-\left(\mathbf{w},\nabla p_{h}\right)_{\Omega}-\left(\mathbf{w},\nabla p_{s}\right)_{\Omega}+\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{h}}\right)_{\Omega}+\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}-\left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u_{h}}\right)_{\Omega}-\left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u_{s}}\right)_{\Omega}-\left(q,\frac{1}{c^{2}}\frac{\partial p_{h}}{\partial t}\right)_{\Omega}-\left(q,\rho\nabla\cdot\mathbf{u_{h}}\right)_{\Omega}-\left(q,\rho\nabla\cdot\mathbf{u_{s}}\right)_{\Omega} & =0
\end{align*}

Integrating by parts the shear stress terms

\begin{eqnarray*}
\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{h}}\right)_{\Omega} & = & -\left(\mathbf{\nabla w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{h}}\right)_{\Omega}+\int_{\Gamma}\mathbf{w}\cdot\left(\mathbb{C}\nabla^s\mathbf{u}_{h}\right)\mathbf{\cdot n}\\
\left(\mathbf{w},\nabla\cdot\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega} & = & -\left(\mathbf{\nabla w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}+\int_{\Gamma}\mathbf{w}\cdot\left(\mathbb{C}\nabla^s\mathbf{u}_{h}\right)\mathbf{\cdot n}\approx-\left(\mathbf{\nabla w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}
\end{eqnarray*}

Doing the same for the pressure terms

\begin{eqnarray*}
\left(\mathbf{w},\nabla p_{h}\right)_{\Omega} & = & -\left(\mathbf{\nabla\cdot w},p_{h}\right)_{\Omega}+\int_{\Gamma}\left(p_{h}\mathbf{w}\right)\mathbf{\cdot n}\\
\left(\mathbf{w},\nabla p_{s}\right)_{\Omega} & = & -\left(\mathbf{\nabla\cdot w},p_{s}\right)_{\Omega}+\int_{\Gamma}\left(p_{h}\mathbf{w}\right)\mathbf{\cdot n\approx}-\left(\mathbf{\nabla\cdot w},p_{s}\right)_{\Omega}
\end{eqnarray*}

Complementary, the subscales mass conservation term is also integrated
by parts as

$
\left(q,\rho\nabla\cdot\mathbf{u_{s}}\right)_{\Omega}=-\left(\nabla q,\rho\mathbf{u_{s}}\right)_{\Omega}+\int_{\Gamma}\left(\rho q\mathbf{n}\right)\mathbf{\cdot u_{s}\approx}-\left(\nabla q,\rho\mathbf{u_{s}}\right)_{\Omega}
$

as well as the subscales convective term

$
\left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u_{s}}\right)_{\Omega}=-\left((\rho(\nabla\cdot\mathbf{a})\mathbf{w}+\rho\mathbf{a}\cdot\nabla\mathbf{w}),\mathbf{u_{s}}\right)_{\Omega}+\int_{\Gamma}(\mathbf{n}\cdot(\mathbf{a}\otimes\mathbf{w}))\mathbf{\cdot u_{s}\approx}-\left((\rho(\nabla\cdot\mathbf{a})\mathbf{w}+\rho\mathbf{a}\cdot\nabla\mathbf{w}),\mathbf{u_{s}}\right)_{\Omega}
$

We now need to introduce a model for the subscales (Algebraic SubGrid
Scales, ASGS):

$ \mathbf{u_{s}} =\tau_{1}\mathbf{R}^{M}(\mathbf{u_{h}},p_{h}) $


$ p_{s} = \tau_{2}\mathbf{R}^{C}(\mathbf{u_{h}},p_{h}) $

being $\tau_{1}$and $\tau_{2}$the stabilization coefficients

$\tau_{1}  =  \left(\frac{c_{1}\mu}{h^{2}}+\frac{c_{2}\rho||\mathbf{a}||}{h}\right)^{-1}$

$\tau_{2}  =  \frac{h^{2}}{c_{1}\tau_{1}}=\mu+\frac{c_{2}\rho h||\mathbf{a}||}{c_{1}}$


where $c_{1}=4$, $c_{2}=2$ and $h$ is the characteristic element
size. Note that the selected subscales model contains higher order
derivatives that vanish when using linear finite elements. Thus, the
final Galerkin residual weak form reads

$\left(\mathbf{w},\rho\mathbf{f}\right)_{\Omega}-\left(\mathbf{w},\rho\frac{\partial\mathbf{u_{h}}}{\partial t}\right)_{\Omega}-\left(\nabla\cdot\mathbf{w},p_{h}\right)_{\Omega}-\left(\nabla\cdot\mathbf{w},p_{s}\right)_{\Omega}-\left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{h}}\right)_{\Omega}-\left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}+\int_{\Gamma}\mathbf{w}\cdot\left(\mathbb{C}\nabla^s\mathbf{u}_{h}-p_{h}\mathbf{I}\right)\mathbf{\cdot n}-\left(\mathbf{w},\rho\mathbf{a}\cdot\nabla\mathbf{u_{h}}\right)_{\Omega}+\left(\rho(\nabla\cdot\mathbf{a})\mathbf{w},\mathbf{u_{s}}\right)_{\Omega}+\left(\rho\mathbf{a}\cdot\nabla\mathbf{w},\mathbf{u_{s}}\right)_{\Omega}-\left(q,\frac{1}{c^{2}}\frac{\partial p_{h}}{\partial t}\right)_{\Omega}-\left(q,\rho\nabla\cdot\mathbf{u_{h}}\right)_{\Omega}+\left(\nabla q,\rho\mathbf{u_{s}}\right)_{\Omega}=0$

Note that the term $\left(\nabla\mathbf{w},\mathbb{\mathbb{C}}\nabla^s\mathbf{u_{s}}\right)_{\Omega}$vanishes
if linear elements are used in the subscale approximation. 

## SYMBOLS TO BE EMPLOYED

Shape functions $N_{I}$ and derivatives $\nabla N_{I}$stored respectively
in a vector $\mathbf{N}$ and a matrix $\mathbf{DN}$. Besides the
following arrays are defined:

- $\mathbf{p}$ such that $p_{I}$ is the current step pressure of node
I
- $\mathbf{p_{n}}$ such that $p_{n,I}$ is the previous step pressure
of node I
- $\mathbf{p_{nn}}$ such that $p_{nn,I}$ is the two previous step
pressure of node I
- $\mathbf{v}$ such that $\mathbf{v_{IK}}$ is the current step velocity
of node I, component K
- $\mathbf{v_{n}}$ such that $\mathbf{v_{n,IK}}$ is the previous step
velocity of node I, component K
- $\mathbf{v_{nn}}$ such that $\mathbf{v_{nn,IK}}$ is the two previous
steps velocity of node I, component K
- $\mathbf{w}$ such that $\mathbf{w_{I}}$ is the linear momentum conservation
test function value I at the considered Gauss point
- $\mathbf{q}$ such that $\mathbf{q_{I}}$ is the mass conservation
test function value I at the considered Gauss point


Values on the gauss points are expressed in terms of such variables
as

- $\mathbf{v\_gauss}:=\mathbf{v}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{p\_gauss}:=\mathbf{p}^{T}\mathbf{N}$ a 1x1 matrix
- $\mathbf{f\_gauss}:=\mathbf{f}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{w\_gauss}:=\mathbf{w}^{T}\mathbf{N}$ a 3x1 matrix
- $\mathbf{q\_gauss}:=\mathbf{q}^{T}\mathbf{N}$ a 1x1 matrix
- $\mathbf{accel\_gauss}:=\left(BDF_{0}\mathbf{V}+BDF_{1}\mathbf{V}_{n}+BDF_{2}\mathbf{V}_{nn}\right)^{T}\mathbf{N}$
a 3x1 matrix
- $\mathbf{pder\_gauss}:=\left(BDF_{0}p+BDF_{1}p_{n}+BDF_{2}p_{nn}\right)\mathbf{N}$
a 1x1 matrix
- $\mathbf{v_{h}:=v^{T}N}$ a 3x1 matrix
- $\mathbf{w_{h}:=w^{T}N}$ a 3x1 matrix
- $p_{h}:=\mathbf{p^{T}N}$ a 1x1 matrix
- $q_{h}:=\mathbf{q^{T}N}$ a 1x1 matrix
- $\mathbf{div\_p}:=\mathbf{\nabla\cdot p}$ a 1x1 matrix
- $\mathbf{div\_v}:=\mathbf{\nabla\cdot v}$ a 1x1 matrix
- $\mathbf{div\_vconv}:=\mathbf{\nabla\cdot vconv}$ a 1x1 matrix
- $\mathbf{grad\_p}:=\mathbf{DN^{T}\cdot p}$ a 3x1 matrix
- $\mathbf{grad\_v}:=\mathbf{DN^{T}\cdot v}$ a 3x3 matrix
- $\mathbf{grad\_vconv}:=\mathbf{DN^{T}\cdot vconv}$ a 3x3 matrix
- $\mathbf{grad\_q}:=\mathbf{DN^{T}\cdot q}$ a 3x1 matrix
- $\mathbf{grad\_w}:=\mathbf{DN^{T}\cdot w}$ a 3x3 matrix
- $\mathbf{grad\_sym\_v}:=$a 3x3 matrix (symmetric gradient in Voigt
form)
- $\mathbf{grad\_sym\_w}:=$a 3x3 matrix (symmetric gradient of the
test function in Voigt form)
- $\mathbf{convective\_term}:=\mathbf{\left(a_{h}\cdot grad\_vh\right)^{T}}$
a 3x1 vector in where $\mathbf{a_{h}}$is the convective velocity.
If the option ``Picard'' linearisation is set, the convective velocity
is defined as a symbol $\mathbf{v_{conv}}$ meaning that it is treated
as a constant in the automatic differenctiation. On the contrary,
if the option ``FullNR'' linearisation is set, the convective velocity
is defined as $(\mathbf{v_{h}}-\mathbf{v_{mesh}})$ implying that
$\mathbf{v_{h}}$ is considered in the automatic differenctiation.
- $\mathbf{stress}:$ 6x1 shear stress vector computed by constitutive
law using the previous iteration database. This stress vector is considered
in the computation of the RHS, which is the residual of the previous
iteration solution. When computing the LHS (RHS derivative w.r.t.
the DOFs), the $\mathbf{stress}$ is substituted by $\mathbf{C*grad\_sym\_v}$
to consider the stress in the automatic differenctiation.



## IMPLEMENTATION

The residual functional implementation is splitted in two pieces


- $\mathbf{rv\_galerkin}$: functional corresponding to the standard
incompressible Navier-Stokes equations. If ``artificial\_compressibility''
is set as True, the artificial compressibility terms are also added
to the functional.
- $\mathbf{rv\_stab}$: functional corresponding to the standard incompressible
Navier-Stokes ASGS stabilization. If ``artificial\_compressibility''
is set as True, the artificial compressibility stabilization terms
are also added to the stabilization functional.
- $\mathbf{rv}=\mathbf{rv\_galerkin}+\mathbf{rv\_stab}$: summation
of functionals. This is the functional used to compute the LHS and
RHS elemental matrices.


For the definition of the subscales, the residuals are splitted in

- $\mathbf{vel\_residual}$: linear momentum conservation equation residual. 
- $\mathbf{mas\_residual}$: mass conservation equation residual. If ```artificial\_compressibility''' is set as True, the artificial compressibility
terms are also added to the residual.


